In [23]:
import pandas as pd
import numpy as np
from math import *
from sklearn import model_selection, metrics, dummy

In [2]:
from sklearn.ensemble import RandomForestRegressor

In [3]:
import pickle

with open("./data/pickle", 'rb') as f:
    data_X, data_Y = pickle.load(f)

In [4]:
# 0 pour les données sans star, 1 pour energystar
data_X = data_X[0]
data_Y = data_Y[0]

In [5]:
col_num = list(data_X.select_dtypes(include=['float', 'int']).columns)
col_num

['DataYear',
 'Latitude',
 'Longitude',
 'YearBuilt',
 'NumberofBuildings',
 'NumberofFloors',
 'PropertyGFATotal',
 'PropertyGFAParking',
 'PropertyGFABuilding(s)',
 'LargestPropertyUseTypeGFA',
 'SiteEUI(kBtu/sf)',
 'SiteEUIWN(kBtu/sf)',
 'SourceEUI(kBtu/sf)',
 'SourceEUIWN(kBtu/sf)',
 'SiteEnergyUseWN(kBtu)',
 'SteamUse(kBtu)',
 'Electricity(kWh)',
 'Electricity(kBtu)',
 'NaturalGas(therms)',
 'NaturalGas(kBtu)',
 'GHGEmissionsIntensity']

In [6]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data_X[col_num], data_Y['Energy'], test_size=0.3)

In [7]:
from sklearn.model_selection import KFold

In [8]:
kf = KFold(n_splits=8)

In [48]:
dum = dummy.DummyRegressor(strategy='median')

score_final = {'R2':0, 'RMSE':0, 'RMSLE':0}

for train, test in kf.split(data_X):
    # Entraînement
    dum.fit(data_X.iloc[train], data_Y.iloc[train])

    # Prédiction sur le jeu de test
    y_pred = dum.predict(data_X.iloc[test])

    
    y_test = data_Y.iloc[test]
    
    # Evaluate
    score = metrics.r2_score(y_test, y_pred)
    print('R2 : ', score)
    score_final['R2'] += abs(score)
    
    score = metrics.mean_squared_error(y_test, y_pred, squared=True)
    #print('RMSE : ', score)
    score_final['RMSE'] += score
    
    score = metrics.mean_squared_log_error(y_test.applymap(lambda x:max(0, x)), y_pred, squared=True)
    #print('RMSLE : ', score)
    score_final['RMSLE'] += score

for k, score in score_final.items():
    print('Score {} final : {}'.format(k, score/kf.get_n_splits()))

R2 :  -0.03935771369497176
R2 :  -0.037885434744178936
R2 :  -0.020967607993529813
R2 :  -0.09025288651906405
R2 :  -0.023981711211913948
R2 :  -0.02139738709119987
R2 :  -0.07694464467844642
R2 :  -0.06193588397876704
Score R2 final : 0.04659040873900898
Score RMSE final : 242323483525120.0
Score RMSLE final : 2.219035920229234


In [49]:
data_X_tmp = data_X[col_num]

score_final = {'R2':0, 'RMSE':0, 'RMSLE':0}

for train, test in kf.split(data_X_tmp):
    regr = RandomForestRegressor(max_depth=5)
    
    # Entraînement
    regr.fit(data_X_tmp.iloc[train], data_Y.iloc[train])

    # Prédiction sur le jeu de test
    y_pred = regr.predict(data_X_tmp.iloc[test])

    
    y_test = data_Y.iloc[test]
    
    # Evaluate
    score = metrics.r2_score(y_test, y_pred)
    print('R2 : ', score)
    score_final['R2'] += abs(score)
    
    score = metrics.mean_squared_error(y_test, y_pred, squared=True)
    #print('RMSE : ', score)
    score_final['RMSE'] += score
    
    score = metrics.mean_squared_log_error(y_test.applymap(lambda x:max(0, x)), y_pred, squared=True)
    #print('RMSLE : ', score)
    score_final['RMSLE'] += score

for k, score in score_final.items():
    print('Score {} final : {}'.format(k, score/kf.get_n_splits()))

R2 :  0.9367300781622593
R2 :  0.9508467824077211
R2 :  0.7973934253651531
R2 :  0.9234314272606028
R2 :  0.9717123246419705
R2 :  0.35062060739756173
R2 :  -1.1645638455118483
R2 :  0.9361240024369618
Score R2 final : 0.8789278116480098
Score RMSE final : 97857438927088.27
Score RMSLE final : 1.02635422468257


In [ ]:
# Entraînement
regr.fit(X_train, y_train)

# Prédiction sur le jeu de test
y_pred = regr.predict(X_test)

# Evaluate
print(metrics.r2_score(y_test, y_pred))